In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [27]:
!ls tmp_img

char


In [12]:
import argparse
import os
import numpy as np
import math

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch
import random, string,os
from PIL import Image, ImageDraw,ImageFont
import tqdm

In [25]:
#W, H = (150,100)
W, H = (55,57)

font = ImageFont.truetype('drive/MyDrive/CS673Data/NotoSansHans-Regular.otf', 55)

if not os.path.exists('./tmp_img'):
   os.makedirs('./tmp_img')
if not os.path.exists('./tmp_img/char'):
   os.makedirs('./tmp_img/char')

for idx, text in enumerate(range(0x4E00,0x8000)):  #0x9FA5
   c_text=chr(text)
#   print(c_text)
   img = Image.new('RGB', (W,H),(255,255,255))
   draw = ImageDraw.Draw(img)
   w, h = draw.textsize(c_text,font=font)
#   print( w ,h )  
   draw.text(((W-w)/2,(H-h)/2), c_text, fill=(0, 0, 0),font=font)
   img.save('./tmp_img/char/'+str(idx)+".png", "PNG")

In [39]:
os.makedirs("images", exist_ok=True)

N_EPOCHS = 200
BATCH_SIZE = 64
LR = .0002
B1 = 0.5
B2 = 0.999
N_CPU = 8
LATENT_DIM = 100
IMG_SIZE = 28
CHANNELS = 1
SAMPLE_INTERVAL = 400


img_shape = (CHANNELS, IMG_SIZE, IMG_SIZE)

cuda = True if torch.cuda.is_available() else False


class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(LATENT_DIM, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.size(0), *img_shape)
        return img


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )

    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        validity = self.model(img_flat)

        return validity


# Loss function
adversarial_loss = torch.nn.BCELoss()

# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()

# Configure data loader
dataloader = torch.utils.data.DataLoader(
    datasets.ImageFolder(root='tmp_img', transform=transforms.Compose([transforms.Grayscale(), transforms.Resize((IMG_SIZE,IMG_SIZE)), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])])),
    batch_size=BATCH_SIZE,
    shuffle=True,
)

# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=LR, betas=(B1, B2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=LR, betas=(B1, B2))

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

# ----------
#  Training
# ----------

loop = tqdm.tqdm(total = N_EPOCHS * len(dataloader), position = 0)

for epoch in range(N_EPOCHS):
    for i, (imgs, _) in enumerate(dataloader):

        # Adversarial ground truths
        valid = Variable(Tensor(imgs.size(0), 1).fill_(1.0), requires_grad=False)
        fake = Variable(Tensor(imgs.size(0), 1).fill_(0.0), requires_grad=False)

        # Configure input
        real_imgs = Variable(imgs.type(Tensor))

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()

        # Sample noise as generator input
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], LATENT_DIM))))

        # Generate a batch of images
        gen_imgs = generator(z)

        # Loss measures generator's ability to fool the discriminator
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Measure discriminator's ability to classify real from generated samples
        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()

        loop.update()
        loop.set_description('Epoch:{}/{}, Batch:{}/{}, D loss:{:.6f}, G Loss:{:.6f}'.format(epoch, N_EPOCHS, i, len(dataloader),
          d_loss.item(), g_loss.item()))

        batches_done = epoch * len(dataloader) + i
        if batches_done % SAMPLE_INTERVAL == 0:
            save_image(gen_imgs.data[:25], "images/%d.png" % batches_done, nrow=5, normalize=True)
    if (epoch % 40 == 39):
      torch.save(generator, f'drive/MyDrive/CS673Data/gen_model_{epoch}_epoch.pt')
      torch.save(discriminator, f'drive/MyDrive/CS673Data/dis_model_{epoch}_epoch.pt')

loop.close()

Epoch:199/200, Batch:199/200, D loss:0.226137, G Loss:5.002018: 100%|██████████| 40000/40000 [25:52<00:00, 25.76it/s]


In [42]:
!ls images

0.png	   15200.png  20400.png  26000.png  31600.png  36800.png  6400.png
10000.png  15600.png  20800.png  26400.png  32000.png  37200.png  6800.png
10400.png  16000.png  21200.png  26800.png  3200.png   37600.png  7200.png
10800.png  1600.png   21600.png  27200.png  32400.png  38000.png  7600.png
11200.png  16400.png  22000.png  27600.png  32800.png  38400.png  8000.png
11600.png  16800.png  22400.png  28000.png  33200.png  38800.png  800.png
12000.png  17200.png  22800.png  2800.png   33600.png  39200.png  8400.png
1200.png   17600.png  23200.png  28400.png  34000.png  39600.png  8800.png
12400.png  18000.png  23600.png  28800.png  34400.png  4000.png   9200.png
12800.png  18400.png  24000.png  29200.png  34800.png  400.png	  9600.png
13200.png  18800.png  2400.png	 29600.png  35200.png  4400.png
13600.png  19200.png  24400.png  30000.png  35600.png  4800.png
14000.png  19600.png  24800.png  30400.png  36000.png  5200.png
14400.png  20000.png  25200.png  30800.png  3600.png   5600.png

In [44]:
!cp images/* drive/MyDrive/CS673Data/SampleImages/